# Automatic Differentiation with `torch.autograd` 自动微分

在训练神经网络时，最常用的算法是反向传播。在该算法中，根据损失函数相对于给定参数的梯度来调整参数（模型权重）。

为了计算这些梯度，`PyTorch` 有一个内置的微分引擎，称为`torch.autograd` 。它支持任何计算图的梯度自动计算。

考虑最简单的一层神经网络，具有输入`x` 、参数`w`和`b`以及一些损失函数。它可以通过以下方式在 PyTorch 中定义：

In [1]:
import torch

x = torch.ones(5)  # input tensor
y = torch.zeros(3)  # expected output
w = torch.randn(5, 3, requires_grad=True)
b = torch.randn(3, requires_grad=True)
z = torch.matmul(x, w)+b
loss = torch.nn.functional.binary_cross_entropy_with_logits(z, y)
print(loss)

tensor(1.6800, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)


## Tensors, Functions and Computational graph 张量、函数和计算图

该代码定义了以下计算图：

![comp-graph](./assets/comp-graph.png)

在这个网络中， w和b是我们需要优化的参数。因此，我们需要能够计算损失函数相对于这些变量的梯度。为此，我们设置这些张量的`requires_grad`属性。

> ### NOTE
> 可以在创建张量时设置`requires_grad`的值，或者稍后使用`x.requires_grad_(True)`方法设置。

我们应用于张量来构造计算图的函数实际上是类`Function`的对象。该对象知道如何向前计算函数，以及如何在向后传播步骤中计算其导数。对反向传播函数的引用存储在张量的grad_fn属性中。您可以在[文档中](https://pytorch.org/docs/stable/autograd.html#function)找到有关`Function`的更多信息。

In [2]:
print(f"Gradient function for z = {z.grad_fn}")
print(f"Gradient function for loss = {loss.grad_fn}")

Gradient function for z = <AddBackward0 object at 0x104879390>
Gradient function for loss = <BinaryCrossEntropyWithLogitsBackward0 object at 0x104879060>


## Computing Gradients 计算梯度

为了优化神经网络中参数的权重，我们需要计算损失函数关于参数的导数，也就是说，我们需要$\frac{\partial \text{loss}}{\partial w}$和$\frac{\partial \text{loss}}{\partial b}$在`x``和`y`的某些固定值下。为了计算这些导数，我们调用`loss.backward()`，然后从`w.grad`和`b.grad`检索值：

In [3]:
loss.backward()
print(w.grad)
print(b.grad)

tensor([[0.0037, 0.3020, 0.3101],
        [0.0037, 0.3020, 0.3101],
        [0.0037, 0.3020, 0.3101],
        [0.0037, 0.3020, 0.3101],
        [0.0037, 0.3020, 0.3101]])
tensor([0.0037, 0.3020, 0.3101])


> ### NOTE
> - 我们只能获取计算图的叶子节点的`grad`属性，这些叶子节点的`requires_grad`属性设置为`True`。对于我们图中的所有其他节点，梯度将不可用。
> - 出于性能原因，我们只能在给定的图上使用一次`backward`执行梯度计算。如果我们需要在同一个图上进行多次`backward`调用，我们需要将`retain_graph=True`传递给backward调用。

## Disabling Gradient Tracking 禁用梯度跟踪

默认情况下，所有`requires_grad=True`的张量都会跟踪其计算历史并支持梯度计算。然而，在某些情况下我们不需要这样做，例如，当我们训练完模型而只想将其应用于某些输入数据时，即我们只想通过网络进行前向计算。我们可以通过用`torch.no_grad()`块包围我们的计算代码来停止跟踪计算：

In [4]:
z = torch.matmul(x, w)+b
print(z.requires_grad)

with torch.no_grad():
    z = torch.matmul(x, w)+b
print(z.requires_grad)

True
False


实现相同结果的另一种方法是在张量上使用`detach()`方法：

In [5]:
z = torch.matmul(x, w)+b
z_det = z.detach()
print(z_det.requires_grad)

False


可能想要禁用梯度跟踪的原因有：
- 将神经网络中的某些参数标记为冻结参数。
- 当仅进行前向传递时加快计算速度，因为对不跟踪梯度的张量进行计算会更有效。

## More on Computational Graphs 有关计算图的更多信息

从概念上讲，`autograd`在由[Function](https://pytorch.org/docs/stable/autograd.html#torch.autograd.Function)对象组成的有向无环图 (DAG) 中保存数据（张量）和所有执行的操作（以及生成的新张量）的记录。在这个 DAG 中，叶子是输入张量，根是输出张量。通过从根到叶追踪该图，可以使用链式法则自动计算梯度。

在前向传递中，autograd 同时执行两件事：
- 运行请求的操作来计算结果张量
- 在 DAG 中维护操作的梯度函数。

当在 DAG 根上调用`.backward()`时，向后传递开始。然后`autograd`：
- 计算每个`.grad_fn`的梯度，
- 将它们累积到各自张量的`.grad`属性中
- 使用链式法则，一直传播到叶子张量。

> ### NOTE
> DAG在PyTorch中是动态的需要注意的重要一点是图是从头开始重新创建的；每次`.backward()`调用后，`autograd`开始填充新图表。这正是允许你在模型中使用控制流语句的原因；如果需要，你可以在每次迭代时更改形状、大小和操作。

## Optional Reading: Tensor Gradients and Jacobian Products 张量梯度和雅可比积

在许多情况下，我们有一个标量损失函数，并且需要计算某些参数的梯度。然而，在某些情况下，输出函数是任意张量。在这种情况下，PyTorch 允许计算所谓的雅可比积，而不是实际的梯度。